# Nash Q-Learning
This tutorial  demonstrates Nash Q-Learning on a zero-sum strategic (non-extensive) game. 

For the payoff matrix, row player is maximizer, coloumn player is minimizer.

Reference: 

Status: TODO

In [1]:
import numpy as np
! pip install pandas
import pandas as pd
# GameMatrix = np.array([[-2,3], [3,-4]])
GameMatrix = np.array([[0,2,-1], [-1,0,1], [1,-1,0]])

Itr = 10000
print('Game matrix: ')
pd.DataFrame(GameMatrix)


You should consider upgrading via the 'pip install --upgrade pip' command.
Game matrix: 


,0,1,2
0,0,2,-1
1,-1,0,1
2,1,-1,0


In [23]:
import sys
sys.path.append("../..")
from mars.env.import_env import make_env
from mars.utils.func import LoadYAML2Dict
from mars.env.import_env import make_env
! pip install pandas
import pandas as pd


yaml_file = '../../mars/confs/mdp_arbitrary_mdp_nxdo2'

args = LoadYAML2Dict(yaml_file, toAttr=True, mergeWith=None)
env = make_env(args)
print(env)
trans_matrices = env.env.trans_prob_matrices
reward_matrices = env.env.trans_prob_matrices
print(np.array(reward_matrices))
for reward_matrix in reward_matrices:
    for r in reward_matrix:
        print(pd.DataFrame(r))

arbitrary_mdp mdp
Load arbitrary_mdp environment in type mdp.
Env observation space: Box(0.0, 12.0, (1,), float32) action space: Discrete(3)
random seed: 83
          0         1         2
0  0.725439  0.004541  0.270020
1  0.019894  0.393816  0.586290
2  0.018132  0.619409  0.362459
3  0.663973  0.203890  0.132137
4  0.488396  0.230743  0.280862
5  0.596318  0.320674  0.083008
6  0.356117  0.142226  0.501657
7  0.099325  0.363757  0.536919
8  0.425645  0.344791  0.229564
          0         1         2
0  0.268496  0.246033  0.485471
1  0.233576  0.317953  0.448471
2  0.234868  0.430463  0.334669
3  0.350786  0.309575  0.339639
4  0.081677  0.356743  0.561580
5  0.192530  0.556450  0.251020
6  0.366567  0.331127  0.302306
7  0.425256  0.530857  0.043887
8  0.390496  0.383753  0.225751
          0         1         2
0  0.362485  0.200677  0.436838
1  0.018321  0.431662  0.550017
2  0.551646  0.203188  0.245165
3  0.416839  0.298845  0.284316
4  0.082880  0.607406  0.309715
5  0.191445

### The equilibrium strategies solved by self-play algorithm

In [37]:
# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

for i in range(Itr):
    # current l is row
    min_id = np.argmin(row_value)
    min_id_list.append(min_id)
    l = GameMatrix[:, min_id] # l is column now
    col_value += np.array(l)
    
    # current l is column
    max_id = np.argmax(col_value)
    max_id_list.append(max_id)
    l = GameMatrix[max_id]  #  l is row now
    row_value += np.array(l)
    
# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])
max_policy = hist/np.sum(hist)
hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1])
min_policy = hist/np.sum(hist)

row_value = row_value / (i+1)  # historical average
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value [lower bound, upper bound]: {row_value}')
print(f'For column player, strategy is {min_policy}, game value [lower bound, upper bound]: {col_value}')


For row player, strategy is [0.25   0.3332 0.4168], game value [lower bound, upper bound]: [0.0836 0.0832 0.0832]
For column player, strategy is [0.3333 0.2499 0.4168], game value [lower bound, upper bound]: [0.083  0.0835 0.0834]


# 2 Fictious Self-Play
Here we try to demonstrate the fictious self-play algorithm (or say a non-neural verison of Neural Fictious Self-Play, NFSP, [2]) on a zero-sum strategic (non-extensive) game.

Note that in original paper of FSP [1], the algorithm of FSP actually uses the supervised learning (SL) and reinforcement learning (RL) language to describe it, quite close to NFSP. SL is for achieving the average policy and RL is for reaching the best response policy. But here we have simple normal-form game with utility written in matrix, so it is a non-neural version. For the average policy, from above FP example we can see, it can be actually directly estimated with the historical frequency of actions for each player. Best response policy is easily calculated since it should be deterministic.

Also note that it is not the XFP algorithm for extensive-form games.

Refereneces:

[1] FSP http://proceedings.mlr.press/v37/heinrich15.html

[2] NFSP https://arxiv.org/abs/1603.01121

In [4]:
import numpy as np
! pip install pandas
import pandas as pd
# GameMatrix = np.array([[-2,3], [3,-4]])
GameMatrix = np.array([[0,2,-1], [-1,0,1], [1,-1,0]])

print('Game matrix: ')
pd.DataFrame(GameMatrix)


Game matrix: 


,0,1,2
0,0,2,-1
1,-1,0,1
2,1,-1,0


In [47]:
# hyperparameters for FSP
Eta = 0.2 # eta=1, only best response policy as FP; eta=0, only average policy; see either [1] or [2]
Avg_warmup = 0.2 # start to get average policy after certain ratio of overall iterations
Itr = 10000

def sample_from_categorical(dist):
    """
    sample once from a categorical distribution, return the entry index.
    dist: should be a list or array of probabilities for a categorical distribution
    """
    return np.argmax(np.random.multinomial(1, dist))

# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

# initialize the average policy as uniform
max_policy = 1./GameMatrix.shape[0]*np.ones(GameMatrix.shape[0])
min_policy = 1./GameMatrix.shape[1]*np.ones(GameMatrix.shape[1])

for i in range(Itr):
    # current l is row
    if i< Avg_warmup or np.isnan(min_policy).any() or sample_from_categorical([1-Eta, Eta]): # if True, the case under Eta, best response (greedy) policy
        min_id = np.argmin(row_value)
        min_id_list.append(min_id)  # the calculation of average policy only uses actions from the best response policy
    else:  # sample action from the average policy
        min_id = sample_from_categorical(min_policy)
    l = GameMatrix[:, min_id] # l is column now
    col_value += np.array(l)
    
    # current l is column
    if i< Avg_warmup or np.isnan(max_policy).any() or sample_from_categorical([1-Eta, Eta]):
        max_id = np.argmax(col_value)
        max_id_list.append(max_id)
    else:
        max_id = sample_from_categorical(max_policy)
    l = GameMatrix[max_id]  #  l is row now
    row_value += np.array(l)
    
    # get the average policy for each player at each iteration
    hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])
    max_policy = hist/np.sum(hist)
    hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1])
    min_policy = hist/np.sum(hist)
    

# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])
max_policy = hist/np.sum(hist)
hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1])
min_policy = hist/np.sum(hist)

row_value = row_value / (i+1)  # historical average
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value [lower bound, upper bound]: {row_value}')
print(f'For column player, strategy is {min_policy}, game value [lower bound, upper bound]: {col_value}')


For row player, strategy is [0.49797776 0.42163802 0.08038423], game value [lower bound, upper bound]: [ 0.0155  0.948  -0.3812]
For column player, strategy is [0.04058116 0.13777555 0.82164329], game value [lower bound, upper bound]: [-0.0518  0.4711 -0.1458]


it seems FSP does not work?

## 1. A modified version, never get best response against the opponent's best response, but only against the average policy.
It works.

In [41]:
# hyperparameters for FSP
Eta = 0.2 # eta=1, only best response policy as FP; eta=0, only average policy; see either [1] or [2]
Avg_warmup = 0.2 # start to get average policy after certain ratio of overall iterations
Itr = 10000  # overall update iteration
N=300  # sample number to evaluate the average policy, since single sample from the average policy will be inaccurate

def sample_from_categorical(dist):
    """
    sample once from a categorical distribution, return the entry index.
    dist: should be a list or array of probabilities for a categorical distribution
    """
    return np.argmax(np.random.multinomial(1, dist))

# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

# initialize the average policy as uniform
max_policy = 1./GameMatrix.shape[0]*np.ones(GameMatrix.shape[0])
min_policy = 1./GameMatrix.shape[1]*np.ones(GameMatrix.shape[1])

for i in range(Itr):
    print(i)
    # current l is row
    if i< Avg_warmup or np.isnan(min_policy).any() or sample_from_categorical([1-Eta, Eta]): # if True, the case under Eta, best response (greedy) policy
        min_id = np.argmin(row_value)
        min_id_list.append(min_id)  # the calculation of average policy only uses actions from the best response policy
    # sample action from the average policy for N times to evaluate the utility of it
    col_value = np.zeros(GameMatrix.shape[1])
    for j in range(N):
        min_id = sample_from_categorical(min_policy)
        l = GameMatrix[:, min_id] # l is column now
        col_value += np.array(l)
#         col_value = (j*col_value + np.array(l))/(j+1)
    col_value = col_value/N
    
    # current l is column
    if i< Avg_warmup or np.isnan(max_policy).any() or sample_from_categorical([1-Eta, Eta]):
        max_id = np.argmax(col_value)
        max_id_list.append(max_id)
    # sample action from the average policy for N times to evaluate the utility of it
    row_value = np.zeros(GameMatrix.shape[0])
    for j in range(N):
        max_id = sample_from_categorical(max_policy)
        l = GameMatrix[max_id]  #  l is row now
        row_value += np.array(l)
#         row_value = (j*row_value + np.array(l))/(j+1)
    row_value = row_value/N

    # get the average policy for each player at each iteration
    hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0], range=[0, GameMatrix.shape[0]-1]) # it's important to provide the range, e.g. for hist([0]) to be [1, 0, 0]
    max_policy = hist/np.sum(hist)
    hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1], range=[0, GameMatrix.shape[1]-1])
    min_policy = hist/np.sum(hist)    

# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0], range=[0, GameMatrix.shape[0]-1])
max_policy = hist/np.sum(hist)
hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1], range=[0, GameMatrix.shape[1]-1])
min_policy = hist/np.sum(hist)

row_value = row_value / (i+1)  # historical average
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value [lower bound, upper bound]: {row_value}')
print(f'For column player, strategy is {min_policy}, game value [lower bound, upper bound]: {col_value}')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069


3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733


5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384


6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055


8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701


## 2. Another modified version, decreasing eta value to 0 to gradually get best response policy against the average policy
It doesn't work.

In [49]:
# hyperparameters for FSP
Eta_ini = 0.2 # eta=1, only best response policy as FP; eta=0, only average policy; see either [1] or [2]
Eta_final = 0
Avg_warmup = 0.2 # start to get average policy after certain ratio of overall iterations
Itr = 10000

def sample_from_categorical(dist):
    """
    sample once from a categorical distribution, return the entry index.
    dist: should be a list or array of probabilities for a categorical distribution
    """
    return np.argmax(np.random.multinomial(1, dist))

# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

# initialize the average policy as uniform
max_policy = 1./GameMatrix.shape[0]*np.ones(GameMatrix.shape[0])
min_policy = 1./GameMatrix.shape[1]*np.ones(GameMatrix.shape[1])

for i in range(Itr):
    Eta = Eta_ini + (Eta_final - Eta_ini)*i/Itr  # decreasing eta
    print(i, Eta)
    # current l is row
    if i< Avg_warmup or np.isnan(min_policy).any() or sample_from_categorical([1-Eta, Eta]): # if True, the case under Eta, best response (greedy) policy
        min_id = np.argmin(row_value)
        min_id_list.append(min_id)  # the calculation of average policy only uses actions from the best response policy
    else:  # sample action from the average policy
        min_id = sample_from_categorical(min_policy)
    l = GameMatrix[:, min_id] # l is column now
    col_value += np.array(l)
    
    # current l is column
    if i< Avg_warmup or np.isnan(max_policy).any() or sample_from_categorical([1-Eta, Eta]):
        max_id = np.argmax(col_value)
        max_id_list.append(max_id)
    else:
        max_id = sample_from_categorical(max_policy)
    l = GameMatrix[max_id]  #  l is row now
    row_value += np.array(l)
    
    # get the average policy for each player at each iteration
    hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])
    max_policy = hist/np.sum(hist)
    hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1])
    min_policy = hist/np.sum(hist)
    

# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])
max_policy = hist/np.sum(hist)
hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1])
min_policy = hist/np.sum(hist)

row_value = row_value / (i+1)  # historical average
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value [lower bound, upper bound]: {row_value}')
print(f'For column player, strategy is {min_policy}, game value [lower bound, upper bound]: {col_value}')


0 0.2
1 0.19998000000000002
2 0.19996
3 0.19994
4 0.19992000000000001
5 0.19990000000000002
6 0.19988
7 0.19986
8 0.19984000000000002
9 0.19982
10 0.1998
11 0.19978
12 0.19976000000000002
13 0.19974
14 0.19972
15 0.19970000000000002
16 0.19968000000000002
17 0.19966
18 0.19964
19 0.19962000000000002
20 0.1996
21 0.19958
22 0.19956000000000002
23 0.19954000000000002
24 0.19952
25 0.1995
26 0.19948000000000002
27 0.19946
28 0.19944
29 0.19942000000000001
30 0.19940000000000002
31 0.19938
32 0.19936
33 0.19934000000000002
34 0.19932
35 0.1993
36 0.19928
37 0.19926000000000002
38 0.19924
39 0.19922
40 0.19920000000000002
41 0.19918000000000002
42 0.19916
43 0.19914
44 0.19912000000000002
45 0.1991
46 0.19908
47 0.19906000000000001
48 0.19904000000000002
49 0.19902
50 0.199
51 0.19898000000000002
52 0.19896
53 0.19894
54 0.19892
55 0.19890000000000002
56 0.19888
57 0.19886
58 0.19884000000000002
59 0.19882000000000002
60 0.1988
61 0.19878
62 0.19876000000000002
63 0.19874
64 0.19872
65 0.19

627 0.18746000000000002
628 0.18744
629 0.18742
630 0.1874
631 0.18738000000000002
632 0.18736
633 0.18734
634 0.18732000000000001
635 0.18730000000000002
636 0.18728
637 0.18726
638 0.18724000000000002
639 0.18722
640 0.1872
641 0.18718
642 0.18716000000000002
643 0.18714
644 0.18712
645 0.18710000000000002
646 0.18708000000000002
647 0.18706
648 0.18704
649 0.18702000000000002
650 0.187
651 0.18698
652 0.18696000000000002
653 0.18694000000000002
654 0.18692
655 0.1869
656 0.18688000000000002
657 0.18686
658 0.18684
659 0.18682
660 0.18680000000000002
661 0.18678
662 0.18676
663 0.18674000000000002
664 0.18672
665 0.1867
666 0.18668
667 0.18666000000000002
668 0.18664
669 0.18662
670 0.18660000000000002
671 0.18658000000000002
672 0.18656
673 0.18654
674 0.18652000000000002
675 0.1865
676 0.18648
677 0.18646000000000001
678 0.18644000000000002
679 0.18642
680 0.1864
681 0.18638000000000002
682 0.18636
683 0.18634
684 0.18632
685 0.18630000000000002
686 0.18628
687 0.18626
688 0.186240

1262 0.17476
1263 0.17474
1264 0.17472000000000001
1265 0.17470000000000002
1266 0.17468
1267 0.17466
1268 0.17464000000000002
1269 0.17462
1270 0.1746
1271 0.17458
1272 0.17456000000000002
1273 0.17454
1274 0.17452
1275 0.17450000000000002
1276 0.17448000000000002
1277 0.17446
1278 0.17444
1279 0.17442000000000002
1280 0.1744
1281 0.17438
1282 0.17436000000000001
1283 0.17434
1284 0.17432
1285 0.1743
1286 0.17428000000000002
1287 0.17426
1288 0.17424
1289 0.17422
1290 0.17420000000000002
1291 0.17418
1292 0.17416
1293 0.17414000000000002
1294 0.17412
1295 0.1741
1296 0.17408
1297 0.17406
1298 0.17404
1299 0.17402
1300 0.17400000000000002
1301 0.17398000000000002
1302 0.17396
1303 0.17394
1304 0.17392000000000002
1305 0.1739
1306 0.17388
1307 0.17386000000000001
1308 0.17384
1309 0.17382
1310 0.1738
1311 0.17378000000000002
1312 0.17376
1313 0.17374
1314 0.17372
1315 0.17370000000000002
1316 0.17368
1317 0.17366
1318 0.17364000000000002
1319 0.17362
1320 0.1736
1321 0.17358
1322 0.1735

1879 0.16242
1880 0.16240000000000002
1881 0.16238
1882 0.16236
1883 0.16234
1884 0.16232000000000002
1885 0.1623
1886 0.16228
1887 0.16226000000000002
1888 0.16224
1889 0.16222
1890 0.1622
1891 0.16218
1892 0.16216
1893 0.16214
1894 0.16212000000000001
1895 0.16210000000000002
1896 0.16208
1897 0.16206
1898 0.16204000000000002
1899 0.16202
1900 0.162
1901 0.16198
1902 0.16196
1903 0.16194
1904 0.16192
1905 0.16190000000000002
1906 0.16188000000000002
1907 0.16186
1908 0.16184
1909 0.16182000000000002
1910 0.1618
1911 0.16178
1912 0.16176000000000001
1913 0.16174
1914 0.16172
1915 0.1617
1916 0.16168
1917 0.16166
1918 0.16164
1919 0.16162
1920 0.16160000000000002
1921 0.16158
1922 0.16156
1923 0.16154000000000002
1924 0.16152
1925 0.1615
1926 0.16148
1927 0.16146
1928 0.16144
1929 0.16142
1930 0.16140000000000002
1931 0.16138000000000002
1932 0.16136
1933 0.16134
1934 0.16132000000000002
1935 0.1613
1936 0.16128
1937 0.16126000000000001
1938 0.16124
1939 0.16122
1940 0.1612
1941 0.1611

2484 0.15032
2485 0.15030000000000002
2486 0.15028
2487 0.15026
2488 0.15024
2489 0.15022000000000002
2490 0.1502
2491 0.15018
2492 0.15016000000000002
2493 0.15014
2494 0.15012
2495 0.1501
2496 0.15008
2497 0.15006
2498 0.15004
2499 0.15002000000000001
2500 0.15000000000000002
2501 0.14998
2502 0.14996
2503 0.14994000000000002
2504 0.14992
2505 0.1499
2506 0.14988
2507 0.14986
2508 0.14984
2509 0.14982
2510 0.14980000000000002
2511 0.14978
2512 0.14976
2513 0.14974
2514 0.14972000000000002
2515 0.1497
2516 0.14968
2517 0.14966000000000002
2518 0.14964
2519 0.14962
2520 0.1496
2521 0.14958
2522 0.14956
2523 0.14954
2524 0.14952000000000001
2525 0.14950000000000002
2526 0.14948
2527 0.14946
2528 0.14944000000000002
2529 0.14942
2530 0.1494
2531 0.14938
2532 0.14936
2533 0.14934
2534 0.14932
2535 0.14930000000000002
2536 0.14928000000000002
2537 0.14926
2538 0.14924
2539 0.14922000000000002
2540 0.1492
2541 0.14918
2542 0.14916000000000001
2543 0.14914
2544 0.14912
2545 0.1491
2546 0.149

...

7773 0.044539999999999996
7774 0.044520000000000004
7775 0.04450000000000001
7776 0.04448000000000002
7777 0.04446
7778 0.04444000000000001
7779 0.04441999999999999
7780 0.04440000000000002
7781 0.04438
7782 0.04436000000000001
7783 0.04433999999999999
7784 0.04432
7785 0.044300000000000006
7786 0.044280000000000014
7787 0.044259999999999994
7788 0.04424
7789 0.04421999999999998
7790 0.04420000000000002
7791 0.04418
7792 0.044160000000000005
7793 0.044139999999999985
7794 0.04411999999999999
7795 0.0441
7796 0.04408000000000001
7797 0.044060000000000016
7798 0.044039999999999996
7799 0.044020000000000004
7800 0.04400000000000001
7801 0.04398000000000002
7802 0.04396
7803 0.04394000000000001
7804 0.04391999999999999
7805 0.04390000000000002
7806 0.04388
7807 0.04386000000000001
7808 0.04383999999999999
7809 0.04382
7810 0.043800000000000006
7811 0.04378000000000001
7812 0.04375999999999999
7813 0.04374
7814 0.04371999999999998
7815 0.04370000000000002
7816 0.04368
7817 0.043660000000000

8243 0.035140000000000005
8244 0.035119999999999985
8245 0.03510000000000002
8246 0.03508
8247 0.03506000000000001
8248 0.03503999999999999
8249 0.035019999999999996
8250 0.035
8251 0.03498000000000001
8252 0.03495999999999999
8253 0.03494
8254 0.03492000000000001
8255 0.034900000000000014
8256 0.034879999999999994
8257 0.03486
8258 0.03484000000000001
8259 0.03481999999999999
8260 0.0348
8261 0.034780000000000005
8262 0.03476000000000001
8263 0.03473999999999999
8264 0.03472
8265 0.03470000000000001
8266 0.034680000000000016
8267 0.034659999999999996
8268 0.034640000000000004
8269 0.034619999999999984
8270 0.03460000000000002
8271 0.03458
8272 0.03456000000000001
8273 0.03453999999999999
8274 0.034519999999999995
8275 0.0345
8276 0.03448000000000001
8277 0.03445999999999999
8278 0.03444
8279 0.034420000000000006
8280 0.034400000000000014
8281 0.034379999999999994
8282 0.03436
8283 0.03434000000000001
8284 0.03431999999999999
8285 0.034300000000000025
8286 0.034280000000000005
8287 0.0

8759 0.02481999999999998
8760 0.024800000000000016
8761 0.024779999999999996
8762 0.024760000000000004
8763 0.024739999999999984
8764 0.024719999999999992
8765 0.0247
8766 0.024680000000000007
8767 0.024660000000000015
8768 0.024639999999999995
8769 0.024620000000000003
8770 0.02460000000000001
8771 0.02458000000000002
8772 0.02456
8773 0.024540000000000006
8774 0.024519999999999986
8775 0.024500000000000022
8776 0.024480000000000002
8777 0.02446000000000001
8778 0.02443999999999999
8779 0.024419999999999997
8780 0.024400000000000005
8781 0.024380000000000013
8782 0.024359999999999993
8783 0.02434
8784 0.02431999999999998
8785 0.024300000000000016
8786 0.024279999999999996
8787 0.024260000000000004
8788 0.024239999999999984
8789 0.02421999999999999
8790 0.0242
8791 0.024180000000000007
8792 0.024160000000000015
8793 0.024139999999999995
8794 0.024120000000000003
8795 0.02410000000000001
8796 0.024080000000000018
8797 0.024059999999999998
8798 0.024040000000000006
8799 0.024019999999999

9260 0.014800000000000008
9261 0.014780000000000015
9262 0.014759999999999995
9263 0.014740000000000003
9264 0.014719999999999983
9265 0.014700000000000019
9266 0.014679999999999999
9267 0.014660000000000006
9268 0.014639999999999986
9269 0.014619999999999994
9270 0.014600000000000002
9271 0.01458000000000001
9272 0.01455999999999999
9273 0.014539999999999997
9274 0.014520000000000005
9275 0.014500000000000013
9276 0.014479999999999993
9277 0.01446
9278 0.014440000000000008
9279 0.014419999999999988
9280 0.014400000000000024
9281 0.014380000000000004
9282 0.014360000000000012
9283 0.014339999999999992
9284 0.01432
9285 0.014300000000000007
9286 0.014280000000000015
9287 0.014259999999999995
9288 0.014240000000000003
9289 0.014219999999999983
9290 0.014200000000000018
9291 0.014179999999999998
9292 0.014160000000000006
9293 0.014139999999999986
9294 0.014119999999999994
9295 0.014100000000000001
9296 0.01408000000000001
9297 0.01405999999999999
9298 0.014039999999999997
9299 0.014020000

9741 0.005180000000000018
9742 0.005159999999999998
9743 0.005140000000000006
9744 0.005119999999999986
9745 0.005100000000000021
9746 0.005080000000000001
9747 0.005060000000000009
9748 0.005039999999999989
9749 0.005019999999999997
9750 0.0050000000000000044
9751 0.004980000000000012
9752 0.004959999999999992
9753 0.00494
9754 0.00491999999999998
9755 0.0049000000000000155
9756 0.0048799999999999955
9757 0.004860000000000003
9758 0.004840000000000011
9759 0.004819999999999991
9760 0.004799999999999999
9761 0.0047800000000000065
9762 0.004760000000000014
9763 0.004739999999999994
9764 0.004720000000000002
9765 0.00470000000000001
9766 0.0046800000000000175
9767 0.0046599999999999975
9768 0.004640000000000005
9769 0.004619999999999985
9770 0.004600000000000021
9771 0.004580000000000001
9772 0.0045600000000000085
9773 0.0045399999999999885
9774 0.004519999999999996
9775 0.004500000000000004
9776 0.004480000000000012
9777 0.004459999999999992
9778 0.0044399999999999995
9779 0.00441999999

In [34]:
import numpy as np
np.histogram([ 2,2,1,], bins=3, range=[0, 2])
# import scipy
# scipy.histogram([0], bins=3)

(array([0, 1, 2]), array([0.        , 0.66666667, 1.33333333, 2.        ]))